### Essential files for Azure Storage

In [33]:
# pip install azure-storage-file-datalake

In [34]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.storage.filedatalake import DataLakeFileClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient

### Liberaries used in code

In [35]:
import numpy as np
import urllib
import random
import json
from sklearn.model_selection import train_test_split
import pathlib
from datetime import datetime, timedelta
import requests
import pandas as pd


### Connect to Storage account

### Function to fetch

In [36]:
def fetch_data_db(username,password):
        global values
        global results
        ##following is example of unauthenticated read... should fail if the db is set to not open read
        url = "http://gapp.agverse.in/api/v1/get-telemetry-data/"
        #response = urllib.request.urlopen(url)
        #this is for authenticated reading
        url_auth=urllib.request.Request(url,headers={"username" : username ,"password" : password})
        response = urllib.request.urlopen(url_auth)
        data = response.read()
        values = json.loads(data)
        results = pd.DataFrame(values)
        return results

[Refer this document for complete steps](https://docs.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-directory-file-acl-python)

### Function to create a dataframe of data obtained from api link

In [37]:
def restAPI_data_to_dataFrame(username,password):
    id = []
    user_id = []
    animal_id = []
    telemetry_data = []
    mac_id = []
#     global timestamp 
    timestamp = []
    results = fetch_data_db(username,password)
    x = results.result 

    for i in x:
        id.append(i['id'])
        user_id.append(i['user_id'])
        animal_id.append(i['animal_id'])
        telemetry_data.append(i['telemetry_data'])
        mac_id.append(i['mac_id'])
        timestamp.append(i['timestamp'])
    _dict_ = {'id' : id,'user_id' : user_id,'animal_id' : animal_id,'telemetry_data' : telemetry_data,'mac_id' : mac_id,'timestamp':timestamp} 
    Data = pd.DataFrame(_dict_)
    return Data

### data_to_storage_acc

In [30]:
def data_to_storage_acc(container_name,my_connection_string,username,password):
    service = BlobServiceClient(account_url="https://sampleagverse.blob.core.windows.net/", credential=storage_account_key)
    
    blob = BlobClient.from_connection_string(conn_str=my_connection_string, container_name=container_name, blob_name=("Data "+str(datetime.datetime.now())))

    data = restAPI_data_to_dataFrame(username,password).to_csv()

    blob.upload_blob(data)
    
    print("Data uploaded successfully")

# Main 

### Connect to storage account

In [39]:
username = ""
password = ""
storage_account_name = ''
storage_account_key = ''
my_connection_string = ''

### Running all the three created functions

In [38]:
# fetch_data_db(username,password)
# restAPI_data_to_dataFrame(username,password)

### Transfer fetched data into our storage account

In [32]:
import datetime

data_to_storage_acc(storage_account_name,my_connection_string,username,password)

Data uploaded successfully
